## Chapter 19: Background Tasks and Hosted Services

Not all work in a web application needs to happen immediately in response to an HTTP request. Tasks like sending emails, processing uploaded files, cleaning up temporary data, or synchronizing with external systems are better handled **in the background**, separate from the request‑response cycle. ASP.NET Core provides a built‑in mechanism for running background tasks: **hosted services**. These are long‑running pieces of code that run in the same process as your web application, managed by the .NET Generic Host. In this chapter, you’ll learn how to create and use hosted services, understand the difference between `IHostedService` and `BackgroundService`, implement timed tasks, handle graceful shutdown, and explore advanced scheduling with libraries like Quartz.NET and Hangfire. By the end, you’ll be able to offload non‑critical work and improve the responsiveness of your application.

### 19.1 Introduction to `IHostedService` and `BackgroundService`

The .NET Generic Host (the foundation of ASP.NET Core applications) is designed to host long‑running services. The interface `IHostedService` defines two methods: `StartAsync` and `StopAsync`. When the host starts, it calls `StartAsync` on all registered hosted services; when the host shuts down, it calls `StopAsync`.

#### `IHostedService` Interface

```csharp
public interface IHostedService
{
    Task StartAsync(CancellationToken cancellationToken);
    Task StopAsync(CancellationToken cancellationToken);
}
```

Implementing `IHostedService` directly gives you full control, but for most background tasks, the abstract base class `BackgroundService` (which implements `IHostedService`) is more convenient. It provides a single method `ExecuteAsync` that you override, and it automatically handles the stopping logic.

#### `BackgroundService` Base Class

```csharp
public abstract class BackgroundService : IHostedService, IDisposable
{
    protected abstract Task ExecuteAsync(CancellationToken stoppingToken);
    // ...
}
```

When the host starts, `StartAsync` calls `ExecuteAsync`. If `ExecuteAsync` completes, the service stops. You typically loop inside `ExecuteAsync` until cancellation is requested.

### 19.2 Implementing a Simple Background Task

Let’s create a simple background service that logs a message every 10 seconds.

```csharp
using Microsoft.Extensions.Hosting;
using Microsoft.Extensions.Logging;
using System;
using System.Threading;
using System.Threading.Tasks;

namespace MyApp.BackgroundServices
{
    public class TimedHostedService : BackgroundService
    {
        private readonly ILogger<TimedHostedService> _logger;
        private readonly TimeSpan _interval = TimeSpan.FromSeconds(10);

        public TimedHostedService(ILogger<TimedHostedService> logger)
        {
            _logger = logger;
        }

        protected override async Task ExecuteAsync(CancellationToken stoppingToken)
        {
            _logger.LogInformation("Timed Hosted Service running.");

            // Use a periodic timer to avoid overlapping executions
            using var timer = new PeriodicTimer(_interval);
            try
            {
                while (await timer.WaitForNextTickAsync(stoppingToken))
                {
                    // Do the work
                    _logger.LogInformation("Timed Hosted Service is working. Time: {time}", DateTimeOffset.Now);
                }
            }
            catch (OperationCanceledException)
            {
                _logger.LogInformation("Timed Hosted Service is stopping.");
            }
        }
    }
}
```

#### Explanation

- `PeriodicTimer` (available in .NET 6+) is an efficient timer that works with `CancellationToken`. Each tick, we perform our work. If the operation takes longer than the interval, the next tick waits until the current one completes, preventing overlapping executions.
- The loop runs until `stoppingToken` is cancelled (e.g., during application shutdown).
- We catch `OperationCanceledException` to handle graceful shutdown.

#### Registering the Service

In `Program.cs`, add the hosted service to the DI container:

```csharp
builder.Services.AddHostedService<TimedHostedService>();
```

Now, when you run the application, you’ll see log messages every 10 seconds.

### 19.3 Using `IHostedService` for Long‑Running Tasks

Sometimes you need a task that runs continuously, like a queue processor that waits for new items. `BackgroundService` is perfect for that.

#### Example: A Queue Processor

Imagine you have a background task that processes messages from a queue. The queue could be an in‑memory collection, a database table, or an external service like RabbitMQ.

First, define a simple in‑memory queue:

```csharp
public interface IBackgroundTaskQueue
{
    void QueueBackgroundWorkItem(Func<CancellationToken, Task> workItem);
    Task<Func<CancellationToken, Task>> DequeueAsync(CancellationToken cancellationToken);
}

public class BackgroundTaskQueue : IBackgroundTaskQueue
{
    private readonly ConcurrentQueue<Func<CancellationToken, Task>> _workItems = new();
    private readonly SemaphoreSlim _signal = new(0);

    public void QueueBackgroundWorkItem(Func<CancellationToken, Task> workItem)
    {
        if (workItem == null) throw new ArgumentNullException(nameof(workItem));
        _workItems.Enqueue(workItem);
        _signal.Release();
    }

    public async Task<Func<CancellationToken, Task>> DequeueAsync(CancellationToken cancellationToken)
    {
        await _signal.WaitAsync(cancellationToken);
        _workItems.TryDequeue(out var workItem);
        return workItem;
    }
}
```

Now create a hosted service that processes this queue:

```csharp
public class QueuedHostedService : BackgroundService
{
    private readonly ILogger<QueuedHostedService> _logger;
    private readonly IBackgroundTaskQueue _taskQueue;

    public QueuedHostedService(IBackgroundTaskQueue taskQueue, ILogger<QueuedHostedService> logger)
    {
        _taskQueue = taskQueue;
        _logger = logger;
    }

    protected override async Task ExecuteAsync(CancellationToken stoppingToken)
    {
        _logger.LogInformation("Queued Hosted Service is starting.");

        while (!stoppingToken.IsCancellationRequested)
        {
            var workItem = await _taskQueue.DequeueAsync(stoppingToken);
            try
            {
                await workItem(stoppingToken);
            }
            catch (Exception ex)
            {
                _logger.LogError(ex, "Error occurred executing work item.");
            }
        }
    }
}
```

Register both the queue and the hosted service:

```csharp
builder.Services.AddSingleton<IBackgroundTaskQueue, BackgroundTaskQueue>();
builder.Services.AddHostedService<QueuedHostedService>();
```

Now from anywhere in your application (e.g., a controller), you can queue work:

```csharp
[ApiController]
[Route("api/[controller]")]
public class JobsController : ControllerBase
{
    private readonly IBackgroundTaskQueue _queue;

    public JobsController(IBackgroundTaskQueue queue)
    {
        _queue = queue;
    }

    [HttpPost]
    public IActionResult Post([FromBody] string jobData)
    {
        _queue.QueueBackgroundWorkItem(async token =>
        {
            // Simulate long-running work
            await Task.Delay(5000, token);
            Console.WriteLine($"Processed: {jobData}");
        });
        return Accepted();
    }
}
```

The request returns immediately with `202 Accepted`, and the work is processed in the background.

### 19.4 Creating Scoped Services Inside Background Tasks

A common pitfall: hosted services are singleton‑scoped. If you need to use scoped services (like `DbContext`), you cannot inject them directly into the constructor. Instead, you must create a scope using `IServiceScopeFactory`.

#### Example: Using DbContext in a Background Service

```csharp
public class DbCleanupService : BackgroundService
{
    private readonly ILogger<DbCleanupService> _logger;
    private readonly IServiceScopeFactory _scopeFactory;
    private readonly TimeSpan _interval = TimeSpan.FromHours(1);

    public DbCleanupService(IServiceScopeFactory scopeFactory, ILogger<DbCleanupService> logger)
    {
        _scopeFactory = scopeFactory;
        _logger = logger;
    }

    protected override async Task ExecuteAsync(CancellationToken stoppingToken)
    {
        using var timer = new PeriodicTimer(_interval);
        while (await timer.WaitForNextTickAsync(stoppingToken))
        {
            try
            {
                await CleanupOldRecordsAsync(stoppingToken);
            }
            catch (Exception ex)
            {
                _logger.LogError(ex, "Error during database cleanup.");
            }
        }
    }

    private async Task CleanupOldRecordsAsync(CancellationToken stoppingToken)
    {
        using var scope = _scopeFactory.CreateScope();
        var dbContext = scope.ServiceProvider.GetRequiredService<AppDbContext>();

        var cutoffDate = DateTime.UtcNow.AddDays(-30);
        var oldLogs = await dbContext.AuditLogs
            .Where(log => log.Timestamp < cutoffDate)
            .ToListAsync(stoppingToken);

        if (oldLogs.Any())
        {
            dbContext.AuditLogs.RemoveRange(oldLogs);
            await dbContext.SaveChangesAsync(stoppingToken);
            _logger.LogInformation("Deleted {count} old audit records.", oldLogs.Count);
        }
    }
}
```

Here, `IServiceScopeFactory` creates a new scope for each execution, ensuring that scoped services (like `DbContext`) are disposed correctly.

### 19.5 Timed Background Tasks with `Timer` or `PeriodicTimer`

We used `PeriodicTimer` in the first example. An alternative is `System.Threading.Timer`, but it’s more manual and requires careful handling of overlapping executions. `PeriodicTimer` is preferred because it integrates with async/await and prevents overlapping.

However, if you’re on an older .NET version, you might use `Timer` with a `ManualResetEvent` or simply `Task.Delay` inside a loop.

```csharp
protected override async Task ExecuteAsync(CancellationToken stoppingToken)
{
    while (!stoppingToken.IsCancellationRequested)
    {
        try
        {
            await DoWorkAsync(stoppingToken);
            await Task.Delay(_interval, stoppingToken);
        }
        catch (OperationCanceledException)
        {
            break;
        }
    }
}
```

This approach is simple but doesn’t guarantee that the interval starts from the end of `DoWorkAsync`. If you need fixed‑rate timing (e.g., every 10 seconds regardless of work duration), you need a more sophisticated timer. `PeriodicTimer` ensures that the next tick occurs after the specified interval from the previous tick, not including the work time.

### 19.6 Graceful Shutdown and Cancellation

Hosted services receive a `CancellationToken` when the application is shutting down. You must respect this token and stop gracefully. In our examples, we pass `stoppingToken` to `PeriodicTimer.WaitForNextTickAsync` and to other async methods. When the token is cancelled, `WaitForNextTickAsync` throws `OperationCanceledException`, which we catch and exit.

For long‑running operations inside the loop, you should also check the token periodically or pass it to methods that support cancellation.

### 19.7 Advanced Scheduling with Quartz.NET or Hangfire

For complex scheduling requirements (cron‑like jobs, persistent job storage, retries, etc.), you might want to use a dedicated scheduling library.

#### Hangfire

Hangfire is a popular library for background jobs in .NET. It provides a persistent storage (SQL Server, Redis, etc.) and a nice dashboard.

**Setup:**

```bash
dotnet add package Hangfire
dotnet add package Hangfire.SqlServer
```

```csharp
builder.Services.AddHangfire(config =>
    config.UseSqlServerStorage(builder.Configuration.GetConnectionString("HangfireConnection")));
builder.Services.AddHangfireServer();

// In app's startup after building
app.UseHangfireDashboard(); // optional dashboard
```

Then you can enqueue jobs from anywhere:

```csharp
BackgroundJob.Enqueue(() => Console.WriteLine("Simple job"));
BackgroundJob.Schedule(() => Console.WriteLine("Delayed job"), TimeSpan.FromDays(1));
RecurringJob.AddOrUpdate("myjob", () => Console.WriteLine("Recurring"), Cron.Hourly);
```

#### Quartz.NET

Quartz.NET is a full‑featured job scheduling library.

```bash
dotnet add package Quartz.Extensions.Hosting
```

```csharp
builder.Services.AddQuartz(q =>
{
    q.UseMicrosoftDependencyInjectionJobFactory();
    q.AddJobAndTrigger<MyJob>("0 0/5 * * * ?"); // every 5 minutes
});
builder.Services.AddQuartzHostedService(q => q.WaitForJobsToComplete = true);
```

Define a job:

```csharp
public class MyJob : IJob
{
    public async Task Execute(IJobExecutionContext context)
    {
        // job logic
    }
}
```

These libraries handle persistence, clustering, and advanced scheduling, making them suitable for enterprise applications.

### 19.8 Monitoring and Logging

Logging in background services is essential. You can inject `ILogger<T>` as usual. Also consider integrating with application health checks to report the status of background workers.

For example, you could implement a health check that verifies that a background task is still running and hasn’t crashed.

```csharp
public class BackgroundServiceHealthCheck : IHealthCheck
{
    private readonly IBackgroundTaskQueue _queue;
    private bool _isHealthy = true;

    public BackgroundServiceHealthCheck(IBackgroundTaskQueue queue)
    {
        _queue = queue;
    }

    public void SetHealthy(bool healthy) => _isHealthy = healthy;

    public Task<HealthCheckResult> CheckHealthAsync(HealthCheckContext context, CancellationToken cancellationToken = default)
    {
        return Task.FromResult(_isHealthy
            ? HealthCheckResult.Healthy()
            : HealthCheckResult.Unhealthy("Background service is not responding."));
    }
}
```

Your hosted service can update the health status periodically.

### 19.9 Putting It All Together: Email Queue Example

Let’s build a complete example: an email queue service. When a user registers, we queue an email instead of sending it immediately. A background service processes the queue.

**EmailService** (simplified)

```csharp
public interface IEmailService
{
    Task SendEmailAsync(string to, string subject, string body);
}

public class SmtpEmailService : IEmailService
{
    private readonly ILogger<SmtpEmailService> _logger;

    public SmtpEmailService(ILogger<SmtpEmailService> logger)
    {
        _logger = logger;
    }

    public async Task SendEmailAsync(string to, string subject, string body)
    {
        // Simulate sending
        _logger.LogInformation("Sending email to {to} with subject {subject}", to, subject);
        await Task.Delay(1000); // simulate network delay
    }
}
```

**EmailQueue** (using the earlier `IBackgroundTaskQueue`)

```csharp
public class EmailQueue
{
    private readonly IBackgroundTaskQueue _queue;

    public EmailQueue(IBackgroundTaskQueue queue)
    {
        _queue = queue;
    }

    public void EnqueueEmail(string to, string subject, string body)
    {
        _queue.QueueBackgroundWorkItem(async token =>
        {
            using var scope = CreateScope(); // we'll need scope for scoped services
            var emailService = scope.ServiceProvider.GetRequiredService<IEmailService>();
            await emailService.SendEmailAsync(to, subject, body);
        });
    }

    private IServiceScope CreateScope()
    {
        // Need access to IServiceScopeFactory – we'll inject it
    }
}
```

To get `IServiceScopeFactory` inside the queue, we need to redesign slightly. Better: let the hosted service create the scope and resolve the email service. The queue item can contain the email data.

Define a class for work items:

```csharp
public class EmailWorkItem
{
    public string To { get; set; }
    public string Subject { get; set; }
    public string Body { get; set; }
}
```

Modify the queue to store these items:

```csharp
public interface IEmailQueue
{
    void Enqueue(EmailWorkItem item);
    Task<EmailWorkItem> DequeueAsync(CancellationToken cancellationToken);
}

public class EmailQueue : IEmailQueue
{
    private readonly ConcurrentQueue<EmailWorkItem> _items = new();
    private readonly SemaphoreSlim _signal = new(0);

    public void Enqueue(EmailWorkItem item)
    {
        _items.Enqueue(item);
        _signal.Release();
    }

    public async Task<EmailWorkItem> DequeueAsync(CancellationToken cancellationToken)
    {
        await _signal.WaitAsync(cancellationToken);
        _items.TryDequeue(out var item);
        return item;
    }
}
```

Now the hosted service:

```csharp
public class EmailProcessingService : BackgroundService
{
    private readonly ILogger<EmailProcessingService> _logger;
    private readonly IEmailQueue _emailQueue;
    private readonly IServiceScopeFactory _scopeFactory;

    public EmailProcessingService(ILogger<EmailProcessingService> logger, IEmailQueue emailQueue, IServiceScopeFactory scopeFactory)
    {
        _logger = logger;
        _emailQueue = emailQueue;
        _scopeFactory = scopeFactory;
    }

    protected override async Task ExecuteAsync(CancellationToken stoppingToken)
    {
        _logger.LogInformation("Email processing service started.");
        while (!stoppingToken.IsCancellationRequested)
        {
            try
            {
                var item = await _emailQueue.DequeueAsync(stoppingToken);
                using var scope = _scopeFactory.CreateScope();
                var emailService = scope.ServiceProvider.GetRequiredService<IEmailService>();
                await emailService.SendEmailAsync(item.To, item.Subject, item.Body);
            }
            catch (OperationCanceledException)
            {
                break;
            }
            catch (Exception ex)
            {
                _logger.LogError(ex, "Error processing email.");
            }
        }
    }
}
```

Register everything:

```csharp
builder.Services.AddSingleton<IEmailQueue, EmailQueue>();
builder.Services.AddScoped<IEmailService, SmtpEmailService>();
builder.Services.AddHostedService<EmailProcessingService>();
```

From a controller:

```csharp
[HttpPost("register")]
public IActionResult Register(RegisterModel model)
{
    // save user...
    _emailQueue.Enqueue(new EmailWorkItem
    {
        To = model.Email,
        Subject = "Welcome!",
        Body = "Thank you for registering."
    });
    return Ok();
}
```

Now emails are sent in the background, improving response time.

### Summary

In this chapter, you’ve learned how to run background tasks in ASP.NET Core using hosted services:

- **`IHostedService`** and **`BackgroundService`** provide the foundation.
- You can implement **timed tasks** with `PeriodicTimer` or `Task.Delay`.
- For long‑running work, use a **queue** and a background processor.
- **Scoped services** require creating a scope via `IServiceScopeFactory`.
- **Graceful shutdown** is handled via `CancellationToken`.
- For advanced scheduling, consider libraries like **Hangfire** or **Quartz.NET**.
- Background tasks are perfect for email sending, data cleanup, report generation, and any work that shouldn’t block HTTP responses.

With these techniques, you can build responsive, scalable applications that handle work efficiently behind the scenes.

**Exercise:**

1. Create a hosted service that logs the current time every 5 seconds.
2. Implement a simple queue for processing long‑running tasks (e.g., image resizing). Enqueue a task from a controller and verify it runs in the background.
3. Modify the email queue example to use a real email library (e.g., MailKit) and test it.
4. (Optional) Install Hangfire and create a recurring job that cleans up old database records daily.

In the next chapter, **"Caching for Performance,"** you’ll learn how to cache data to reduce database load and improve response times. We’ll cover in‑memory caching, distributed caching with Redis, response caching, and cache invalidation strategies.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='18. working_with_real_time_features.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='20. caching_for_performance.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
